In [ ]:
!pip install git+https://github.com/speechbrain/speechbrain.git

  Cloning https://github.com/speechbrain/speechbrain.git to /tmp/pip-req-build-3rzv8z2v
  Running command git clone --filter=blob:none --quiet https://github.com/speechbrain/speechbrain.git /tmp/pip-req-build-3rzv8z2v
  Resolved https://github.com/speechbrain/speechbrain.git to commit 5beaece0b4cdcce18f303cbb1f5f04b0a879dca2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvid

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/MultiModal_Project/audio"

Mounted at /content/drive


In [ ]:
import os

In [ ]:
from speechbrain.inference.interfaces import foreign_class
classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")
X, y = [], []
for file_name in os.listdir(path):
    X.append(path+ '/' + file_name)
    y.append(file_name.split("_")[2])


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
predicted = []
for x in X:
  out_prob, score, index, text_lab = classifier.classify_file(x)
  predicted.append(text_lab[0])

In [ ]:
predicted = [a.upper() for a in predicted]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y, predicted)

0.33015318462778825

In [ ]:
import keras
import librosa
import numpy as np
import pandas as pd


class LivePredictions:

    def __init__(self, file):
        self.file = file
        self.path = 'Emotion_Voice_Detection_Model.h5'
        self.loaded_model = keras.models.load_model(self.path)

    def make_predictions(self):
        X, sample_rate = librosa.load(self.file)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        livedf2 = mfccs
        livedf2= pd.DataFrame(data=livedf2)
        livedf2 = livedf2.stack().to_frame()
        # data, sampling_rate = librosa.load(self.file)
        # mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
        x = np.expand_dims(livedf2, axis=2)
        x = np.expand_dims(x, axis=0)
        predictions = self.loaded_model.predict(x)
        predictions = np.argmax(predictions,axis=1)
        return self.convert_class_to_emotion(predictions)

    @staticmethod
    def convert_class_to_emotion(pred):

        label_conversion = {'0': 'neutral',
                            '1': 'calm',
                            '2': 'happy',
                            '3': 'sad',
                            '4': 'angry',
                            '5': 'fearful',
                            '6': 'disgust',
                            '7': 'surprised'}

        for key, value in label_conversion.items():
            if int(key) == pred:
                label = value
        return label


In [ ]:
predicted_2, ctr = [], 0
for x in X:
  ctr+=1
  pred = LivePredictions(x)
  predicted_2.append(pred.make_predictions())
  if not ctr%100:
    print(ctr)

1/1 [==============================] - 0s 105ms/step


1/1 [==============================] - 0s 108ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 107ms/step
2500
1/1 [==============================] - 0s 76ms/step
2600
1/1 [==============================] - 0s 99ms/step
2700
1/1 [==============================] - 0s 132ms/step
2800
1/1 [==============================] - 0s 94ms/step
2900
1/1 [==============================] - 0s 116ms/step
3000
1/1 [==============================] - 0s 85ms/step
3100
1/1 [==============================] - 0s 90ms/step
3200
1/1 [==============================] - 0s 80ms/step
3300
1/1 [==============================] - 0s 95ms/step
3400
1/1 [==============================] - 0s 93ms/step
3500
1/1 [==============================] - 0s 82ms/step
3600
1/1 [==============================] - 0s 79ms/step
3700
1/1 [==============================] - 0s 186ms/step
3800
1/1 [==============================] - 0s 111ms/step
3900
1/1 [==============================] - 0s 110ms/step
4000
1/1 [==========================

In [ ]:
print(predicted_2)
print(y)
mp = {'disgust': 'DIS', 'sad': 'SAD', 'fearful': 'FEA', 'happy': 'HAP', 'neutral': 'NEU', 'angry': 'ANG', 'calm': 'NEU'}
predicted_2 = [mp[a] for a in predicted_2]

accuracy_score(y, predicted_2)

['DIS', 'DIS', 'HAP', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'HAP', 'DIS', 'SAD', 'HAP', 'DIS', 'DIS', 'DIS', 'SAD', 'SAD', 'SAD', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'SAD', 'SAD', 'FEA', 'HAP', 'SAD', 'DIS', 'SAD', 'HAP', 'DIS', 'SAD', 'SAD', 'DIS', 'HAP', 'DIS', 'DIS', 'NEU', 'DIS', 'SAD', 'SAD', 'SAD', 'DIS', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'SAD', 'DIS', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'SAD', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'SAD', 'SAD', 'DIS', 'DIS', 'DIS', 'SAD', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'SAD', 'SAD', 'DIS', 'DIS', 'SAD', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS', 'SAD', 'DIS', 'DIS', 'SAD', 'SAD', 'SAD', 'SAD', 'SAD', 'DIS', 'DIS', 'DIS', 'DIS', 'DIS'

0.20088685837140555

In [ ]:
!pip install pydub
import numpy as np
import librosa
from sklearn.cluster import KMeans
import soundfile as sf
from pydub import AudioSegment

# Function to extract features from the speech signal
def extract_features(signal, sr=22050, frame_length=2048, hop_length=512, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc, n_fft=frame_length, hop_length=hop_length)
    return mfccs.T

# Function to perform vector quantization
def vector_quantization(features, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features)
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_
    return centroids, labels

# Function to reconstruct audio from quantized features
def reconstruct_audio(centroids, labels, hop_length=512):
    reconstructed_features = centroids[labels]
    reconstructed_audio = librosa.feature.inverse.mfcc_to_audio(reconstructed_features.T, hop_length=hop_length)
    return reconstructed_audio


def quantize_audio(file_path):
  signal, sr = librosa.load(file_path, sr=None)
  signal = np.array(signal)
  # Extract features
  features = extract_features(signal)

  # Perform vector quantization
  num_clusters = 16
  centroids, labels = vector_quantization(features, num_clusters)

  # Reconstruct audio
  reconstructed_audio = reconstruct_audio(centroids, labels)

  return reconstructed_audio

In [ ]:
import numpy as np
import tensorflow as tf
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder

# Function to extract MFCC features from an audio file
def extract_mfccs(file, sr=22050, n_mfcc=13):

    X, sample_rate = librosa.load(file)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    return mfccs.flatten()

# List to store MFCC features and corresponding speaker labels
x = []
y = []
dicti = {"ANG": 0, "SAD": 1, "HAP": 2, "DIS": 3, "FEA": 4, "NEU": 5}
for file in os.listdir(path):
    emo = dicti[file.split("_")[2]]
    y.append(emo)
    X, sample_rate = librosa.load(path+ '/'+file)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    x.append(mfccs.flatten())


# Step 3: Train Emotion Recognition Model
X_train, X_test, y_train, y_test = train_test_split(np.array(x), np.array(y), test_size=0.4, random_state=42, stratify = y)
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# Reconstructed audio testing
predicted_reco1 = []
ctr = 0
for file in os.listdir(path):
  ctr+=1
  output_path = 'temp.wav'
  reconstructed_audio = quantize_audio(path+ '/'+file)
  sf.write(output_path, reconstructed_audio, 48000, 'PCM_24')
  X, sample_rate = librosa.load(path+ '/'+file)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
  # live_prediction = LivePredictions(file=output_path)
  # live_prediction.loaded_model.summary()
  predicted_reco1.append(rf_classifier.predict([mfccs.flatten()]))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/py

In [ ]:
predicted_reco1 = [a[0] for a in predicted_reco1]
from sklearn.metrics import accuracy_score
accuracy_score(y, predicted_reco1)

0.777344799785004

In [ ]:
!pip install git+https://github.com/speechbrain/speechbrain.git@develop


  Cloning https://github.com/speechbrain/speechbrain.git (to revision develop) to /tmp/pip-req-build-b5pjydg2
  Running command git clone --filter=blob:none --quiet https://github.com/speechbrain/speechbrain.git /tmp/pip-req-build-b5pjydg2
  Resolved https://github.com/speechbrain/speechbrain.git to commit 5beaece0b4cdcce18f303cbb1f5f04b0a879dca2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [ ]:
from collections import defaultdict
from speechbrain.inference.speaker import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")
mp = defaultdict(list)
original, constructed = [], []
ctr = 0
for file in os.listdir(path):
  ctr+=1
  output_path = 'temp.wav'
  reconstructed_audio = quantize_audio(path+ '/'+file)
  sf.write(output_path, reconstructed_audio, 48000, 'PCM_24')
  constructed.append(verification.verify_files(path+'/'+file, output_path)[1])
  sp = file.split("_")[0]
  for a in mp[sp]:
    original.append(verification.verify_files(a, path+'/'+file)[1] )
  mp[sp].append(path+'/'+file)
  if ctr == 200:
    break

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8